# Сравнение

In [1]:
import time
import pandas as pd
from datetime import datetime

# Параметры задачи
NUM_BUSES = 8
ROUT_TIME = 60
START_TIME = 6 * 60
END_TIME = 27 * 60
DRIVE_CHANGE_TIME = 15
PEAK_HOURS = [(7 * 60, 9 * 60), (17 * 60, 19 * 60)]
DRIVER_A_MAX_HOURS = 8 * 60
DRIVER_B_MAX_HOURS = 12 * 60

# Форматирование времени
def format_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    if hours >= 24:
        hours -= 24
        return f"{hours:02d}:{mins:02d} (следующий день)"
    return f"{hours:02d}:{mins:02d}"

# Генерация расписания для одного автобуса
def generate_bus_schedule(bus_id, driver_type):
    schedule = []
    current_time = START_TIME
    max_hours = DRIVER_A_MAX_HOURS if driver_type == "A" else DRIVER_B_MAX_HOURS

    while current_time < END_TIME:
        shift_start = current_time
        shift_end = min(shift_start + ROUT_TIME, END_TIME)

        schedule.append({
            "Bus": bus_id,
            "Driver Type": driver_type,
            "Start_min": shift_start,
            "End_min": shift_end,
            "Start Formatted": format_time(shift_start),
            "End Formatted": format_time(shift_end),
            "Shift Duration": ROUT_TIME,
        })

        current_time = shift_end + DRIVE_CHANGE_TIME
        if current_time - START_TIME >= max_hours:
            break

    return schedule

# Генерация полного расписания
def generate_full_schedule():
    schedules = []
    for bus_id in range(1, NUM_BUSES + 1):
        driver_type = "A" if bus_id % 2 == 0 else "B"
        schedules.extend(generate_bus_schedule(bus_id, driver_type))
    return pd.DataFrame(schedules)

# Подсчет автобусов в часы пик
def count_buses_in_peak(schedule, peak_hours):
    start_peak, end_peak = peak_hours
    buses_in_peak = 0
    for _, row in schedule.iterrows():
        if row['Start_min'] < end_peak and row['End_min'] > start_peak:
            buses_in_peak += 1
    return buses_in_peak

# Оценка расписания
def evaluate_schedule(schedule):
    morning_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[0])
    evening_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[1])
    return morning_peak_buses + evening_peak_buses

# Генетический алгоритм
def genetic_algorithm(iterations=100, population_size=10):
    full_schedule = generate_full_schedule()
    best_schedule = None
    best_score = -1

    for _ in range(iterations):
        population = [
            full_schedule.sample(frac=1).reset_index(drop=True) for _ in range(population_size)
        ]

        for schedule in population:
            score = evaluate_schedule(schedule)
            if score > best_score:
                best_schedule = schedule
                best_score = score

    return best_schedule, best_score

# Алгоритм в лоб
def brute_force_schedule():
    full_schedule = generate_full_schedule()
    best_schedule = None
    best_score = -1

    for _ in range(100):  # Перебор 100 случайных вариантов
        random_schedule = full_schedule.sample(frac=1).reset_index(drop=True)
        score = evaluate_schedule(random_schedule)

        if score > best_score:
            best_schedule = random_schedule
            best_score = score

    return best_schedule, best_score

# Сравнение двух алгоритмов
def compare_algorithms():
    print("Запуск генетического алгоритма...")
    start_time = time.time()
    genetic_result, genetic_score = genetic_algorithm()
    genetic_time = time.time() - start_time
    print(f"Генетический алгоритм: {genetic_score} автобусов в часы пик, время выполнения: {genetic_time:.4f} сек")

    print("\nЗапуск алгоритма в лоб...")
    start_time = time.time()
    brute_force_result, brute_force_score = brute_force_schedule()
    brute_force_time = time.time() - start_time
    print(f"Алгоритм в лоб: {brute_force_score} автобусов в часы пик, время выполнения: {brute_force_time:.4f} сек")

    # Сравнение результатов
    print("\nСравнение:")
    print(f"Генетический алгоритм: {genetic_score} (время: {genetic_time:.4f} сек)")
    print(f"Алгоритм в лоб: {brute_force_score} (время: {brute_force_time:.4f} сек)")

# Запуск сравнения
compare_algorithms()


Запуск генетического алгоритма...
Генетический алгоритм: 20 автобусов в часы пик, время выполнения: 4.8209 сек

Запуск алгоритма в лоб...
Алгоритм в лоб: 20 автобусов в часы пик, время выполнения: 0.5083 сек

Сравнение:
Генетический алгоритм: 20 (время: 4.8209 сек)
Алгоритм в лоб: 20 (время: 0.5083 сек)


# Генетический 1

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Параметры задачи
NUM_BUSES = 8
ROUT_TIME = 60
START_TIME = 6 * 60
END_TIME = 27 * 60
DRIVE_CHANGE_TIME = 15
PEAK_HOURS = [(7 * 60, 9 * 60), (17 * 60, 19 * 60)]
DRIVER_A_MAX_HOURS = 8 * 60
DRIVER_B_MAX_HOURS = 12 * 60
POPULATION_SIZE = 10
GENERATIONS = 50
MUTATION_RATE = 0.1

# Форматирование времени для удобного чтения
def format_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    if hours >= 24:
        hours -= 24
        return f"{hours:02d}:{mins:02d} (следующий день)"
    return f"{hours:02d}:{mins:02d}"

# Подсчет автобусов, работающих в часы пик
def count_buses_in_peak(schedule, peak_hours):
    start_peak, end_peak = peak_hours
    buses_in_peak = 0
    for _, row in schedule.iterrows():
        if row['Start_min'] < end_peak and row['End_min'] > start_peak:
            buses_in_peak += 1
    return buses_in_peak

# Генерация полного расписания для всех автобусов
def generate_full_schedule():
    schedules = []
    for bus_id in range(1, NUM_BUSES + 1):
        driver_type = "A" if bus_id % 2 == 0 else "B"
        max_hours = DRIVER_A_MAX_HOURS if driver_type == "A" else DRIVER_B_MAX_HOURS
        current_time = START_TIME

        while current_time < END_TIME:
            shift_start = current_time
            shift_end = min(shift_start + ROUT_TIME, END_TIME)

            schedules.append({
                "Bus": bus_id,
                "Driver Type": driver_type,
                "Start_min": shift_start,
                "End_min": shift_end,
                "Start Formatted": format_time(shift_start),
                "End Formatted": format_time(shift_end),
                "Shift Duration": ROUT_TIME,
            })

            current_time = shift_end + DRIVE_CHANGE_TIME

            if current_time - START_TIME >= max_hours:
                break

    return pd.DataFrame(schedules)

# Оценка расписания
def evaluate_schedule(schedule):
    morning_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[0])
    evening_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[1])
    return morning_peak_buses + evening_peak_buses

# Создание начальной популяции
def initialize_population():
    full_schedule = generate_full_schedule()
    return [full_schedule.sample(frac=1).reset_index(drop=True) for _ in range(POPULATION_SIZE)]

# Отбор лучших расписаний
def select_best(population, scores):
    sorted_population = [x for _, x in sorted(zip(scores, population), key=lambda pair: pair[0], reverse=True)]
    return sorted_population[:POPULATION_SIZE // 2]

# Скрещивание расписаний
def crossover(schedule1, schedule2):
    point = random.randint(1, len(schedule1) - 2)
    child1 = pd.concat([schedule1[:point], schedule2[point:]]).reset_index(drop=True)
    child2 = pd.concat([schedule2[:point], schedule1[point:]]).reset_index(drop=True)
    return child1, child2

# Мутация расписания
def mutate(schedule):
    if random.random() < MUTATION_RATE:
        idx = random.randint(0, len(schedule) - 1)
        schedule.iloc[idx], schedule.iloc[(idx + 1) % len(schedule)] = (
            schedule.iloc[(idx + 1) % len(schedule)],
            schedule.iloc[idx],
        )
    return schedule

# Генетический алгоритм
def genetic_algorithm():
    population = initialize_population()

    for generation in range(GENERATIONS):
        scores = [evaluate_schedule(schedule) for schedule in population]

        # Отбор лучших
        best_population = select_best(population, scores)

        # Скрещивание
        next_population = []
        while len(next_population) < POPULATION_SIZE:
            parents = random.sample(best_population, 2)
            children = crossover(parents[0], parents[1])
            next_population.extend(children)

        # Мутация
        population = [mutate(schedule) for schedule in next_population]

    # Возврат лучшего расписания
    final_scores = [evaluate_schedule(schedule) for schedule in population]
    best_index = final_scores.index(max(final_scores))
    return population[best_index], max(final_scores)

# Выполнение алгоритма
schedule, score = genetic_algorithm()

# Вывод результатов
print("Оптимальное расписание автобусов (генетический алгоритм):")
print(schedule)
print(f"Количество автобусов в часы пик: {score}")


Оптимальное расписание автобусов (генетический алгоритм):
    Bus Driver Type  Start_min  End_min Start Formatted End Formatted  \
0     3           B        360      420           06:00         07:00   
1     3           B        510      570           08:30         09:30   
2     5           B        735      795           12:15         13:15   
3     7           B        960     1020           16:00         17:00   
4     3           B        510      570           08:30         09:30   
..  ...         ...        ...      ...             ...           ...   
63    3           B        810      870           13:30         14:30   
64    6           A        510      570           08:30         09:30   
65    7           B        735      795           12:15         13:15   
66    4           A        510      570           08:30         09:30   
67    1           B        960     1020           16:00         17:00   

    Shift Duration  
0               60  
1               60  
2 

# Генетический 2

In [3]:
import random 
import pandas as pd
from datetime import datetime, timedelta 

#Параметры задачи
NUM_BUSES = 8
ROUT_TIME = 60
START_TIME = 6 * 60
END_TIME = 27 * 60 
DRIVE_CHANGE_TIME = 15
PEAK_HOURS = [(7 * 60, 9 * 60), (17 * 60, 19 * 60)]  # Часы пик
LOAD_PIK = 0.7
LOAD_OFF_PEAK = 0.3

#Время работы (день и ночь)
work_day_start = datetime.strptime("06:00", "%H:%M")
work_day_end = datetime.strptime("22:00", "%H:%M")
night_start = datetime.strptime("22:00", "%H:%M")
night_end = datetime.strptime("03:00", "%H:%M") + timedelta(days=1)


#Временные параметры для водителей
DRIVER_A_MAX_HOURS = 8 * 60
DRIVER_B_MAX_HOURS = 12 * 60
BREAK_MIN = 15

# Форматирование времени для удобного чтения
def format_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    if hours >= 24:  # Перенос на следующий день
        hours -= 24
        return f"{hours:02d}:{mins:02d} (следующий день)"
    return f"{hours:02d}:{mins:02d}"

# Генерация расписания для одного автобуса
def generate_bus_schedule(bus_id, driver_type):
    schedule = []
    current_time = START_TIME
    max_hours = DRIVER_A_MAX_HOURS if driver_type == "A" else DRIVER_B_MAX_HOURS

    while current_time < END_TIME:
        shift_start = current_time
        shift_end = shift_start + ROUT_TIME
        if shift_end > END_TIME:
            shift_end = END_TIME

        # Добавляем смену автобуса
        schedule.append({
            "Bus": bus_id,
            "Driver Type": driver_type,
            "Start": format_time(shift_start),
            "End": format_time(shift_end),
            "Shift Duration": ROUT_TIME,
        })

        # Учитываем время смены водителя
        current_time = shift_end + DRIVE_CHANGE_TIME

        # Проверяем максимальное время работы водителя
        if current_time - shift_start >= max_hours:
            break

    return schedule

# Расписание всех автобусов
def generate_full_schedule(num_buses):
    full_schedule = []
    for bus_id in range(1, num_buses + 1):
        # Чередуем типы водителей для каждого автобуса
        driver_type = "A" if bus_id % 2 == 0 else "B"
        bus_schedule = generate_bus_schedule(bus_id, driver_type)
        full_schedule.extend(bus_schedule)
    return pd.DataFrame(full_schedule)

# Расчет автобусов в часы пик
def count_buses_in_peak(schedule, peak_hours):
    start_peak, end_peak = peak_hours
    
    # Преобразуем время "Start" и "End" в минуты от начала суток
    def time_to_minutes(time_str):
        parts = time_str.split(':')
        hours, minutes = int(parts[0]), int(parts[1].split()[0])  # Убираем возможное "(следующий день)"
        if "следующий день" in time_str:
            hours += 24
        return hours * 60 + minutes
    
    schedule['Start_min'] = schedule['Start'].apply(time_to_minutes)
    schedule['End_min'] = schedule['End'].apply(time_to_minutes)
    
    # Выбираем записи, попадающие в час пик
    peak_schedule = schedule[
        (schedule['Start_min'] < end_peak) & 
        (schedule['End_min'] > start_peak)
    ]
    return len(peak_schedule)

# Генерация расписания
schedule = generate_full_schedule(NUM_BUSES)

# Подсчет автобусов в часы пик
morning_peak = count_buses_in_peak(schedule, PEAK_HOURS[0])
evening_peak = count_buses_in_peak(schedule, PEAK_HOURS[1])

# Вывод результатов
print("Полное расписание автобусов:")
print(schedule)

print(f"Количество автобусов в утренний час пик: {morning_peak}")
print(f"Количество автобусов в вечерний час пик: {evening_peak}")


Полное расписание автобусов:
     Bus Driver Type                   Start                     End  \
0      1           B                   06:00                   07:00   
1      1           B                   07:15                   08:15   
2      1           B                   08:30                   09:30   
3      1           B                   09:45                   10:45   
4      1           B                   11:00                   12:00   
..   ...         ...                     ...                     ...   
131    8           A                   21:00                   22:00   
132    8           A                   22:15                   23:15   
133    8           A                   23:30  00:30 (следующий день)   
134    8           A  00:45 (следующий день)  01:45 (следующий день)   
135    8           A  02:00 (следующий день)  03:00 (следующий день)   

     Shift Duration  Start_min  End_min  
0                60        360      420  
1                60   

# В лоб

In [4]:
import pandas as pd
from datetime import datetime, timedelta

# Параметры задачи
NUM_BUSES = 8
ROUT_TIME = 60
START_TIME = 6 * 60
END_TIME = 27 * 60
DRIVE_CHANGE_TIME = 15
PEAK_HOURS = [(7 * 60, 9 * 60), (17 * 60, 19 * 60)]
DRIVER_A_MAX_HOURS = 8 * 60
DRIVER_B_MAX_HOURS = 12 * 60

# Форматирование времени для удобного чтения
def format_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    if hours >= 24:
        hours -= 24
        return f"{hours:02d}:{mins:02d} (следующий день)"
    return f"{hours:02d}:{mins:02d}"

# Подсчет автобусов, работающих в часы пик
def count_buses_in_peak(schedule, peak_hours):
    start_peak, end_peak = peak_hours
    buses_in_peak = 0
    for _, row in schedule.iterrows():
        if row['Start_min'] < end_peak and row['End_min'] > start_peak:
            buses_in_peak += 1
    return buses_in_peak

# Генерация полного расписания для всех автобусов
def generate_full_schedule():
    schedules = []
    for bus_id in range(1, NUM_BUSES + 1):
        driver_type = "A" if bus_id % 2 == 0 else "B"
        max_hours = DRIVER_A_MAX_HOURS if driver_type == "A" else DRIVER_B_MAX_HOURS
        current_time = START_TIME

        while current_time < END_TIME:
            shift_start = current_time
            shift_end = min(shift_start + ROUT_TIME, END_TIME)

            schedules.append({
                "Bus": bus_id,
                "Driver Type": driver_type,
                "Start_min": shift_start,
                "End_min": shift_end,
                "Start Formatted": format_time(shift_start),
                "End Formatted": format_time(shift_end),
                "Shift Duration": ROUT_TIME,
            })

            current_time = shift_end + DRIVE_CHANGE_TIME

            if current_time - START_TIME >= max_hours:
                break

    return pd.DataFrame(schedules)

# Оценка расписания
def evaluate_schedule(schedule):
    morning_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[0])
    evening_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[1])
    return morning_peak_buses + evening_peak_buses

# Алгоритм полного перебора
def brute_force_schedule():
    best_schedule = None
    best_score = -1

    # Генерация всех возможных комбинаций
    full_schedule = generate_full_schedule()

    # Проверяем все возможные расписания
    for _ in range(100):  # Пример ограниченного перебора
        random_schedule = full_schedule.sample(frac=1).reset_index(drop=True)  # Перемешиваем
        score = evaluate_schedule(random_schedule)

        if score > best_score:
            best_schedule = random_schedule
            best_score = score

    return best_schedule, best_score

# Выполнение алгоритма
schedule, score = brute_force_schedule()

# Вывод результатов
print("Оптимальное расписание автобусов:")
print(schedule)
print(f"Количество автобусов в часы пик: {score}")


Оптимальное расписание автобусов:
    Bus Driver Type  Start_min  End_min Start Formatted End Formatted  \
0     1           B        960     1020           16:00         17:00   
1     3           B        360      420           06:00         07:00   
2     3           B        435      495           07:15         08:15   
3     4           A        810      870           13:30         14:30   
4     1           B       1035     1095           17:15         18:15   
..  ...         ...        ...      ...             ...           ...   
63    1           B        660      720           11:00         12:00   
64    5           B        660      720           11:00         12:00   
65    7           B        585      645           09:45         10:45   
66    1           B        810      870           13:30         14:30   
67    7           B        735      795           12:15         13:15   

    Shift Duration  
0               60  
1               60  
2               60  
3    

# Сравнение(такое же как и первое)

In [5]:
import time
import pandas as pd
from datetime import datetime, timedelta

# Параметры задачи (общие для обоих алгоритмов)
NUM_BUSES = 8
ROUT_TIME = 60
START_TIME = 6 * 60
END_TIME = 27 * 60
DRIVE_CHANGE_TIME = 15
PEAK_HOURS = [(7 * 60, 9 * 60), (17 * 60, 19 * 60)]
DRIVER_A_MAX_HOURS = 8 * 60
DRIVER_B_MAX_HOURS = 12 * 60

# Форматирование времени для удобного чтения
def format_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    if hours >= 24:
        hours -= 24
        return f"{hours:02d}:{mins:02d} (следующий день)"
    return f"{hours:02d}:{mins:02d}"

# Подсчет автобусов в часы пик
def count_buses_in_peak(schedule, peak_hours):
    start_peak, end_peak = peak_hours
    buses_in_peak = 0
    for _, row in schedule.iterrows():
        if row['Start_min'] < end_peak and row['End_min'] > start_peak:
            buses_in_peak += 1
    return buses_in_peak

# Генерация расписания для одного автобуса
def generate_bus_schedule(bus_id, driver_type):
    schedule = []
    current_time = START_TIME
    max_hours = DRIVER_A_MAX_HOURS if driver_type == "A" else DRIVER_B_MAX_HOURS

    while current_time < END_TIME:
        shift_start = current_time
        shift_end = min(shift_start + ROUT_TIME, END_TIME)

        schedule.append({
            "Bus": bus_id,
            "Driver Type": driver_type,
            "Start_min": shift_start,
            "End_min": shift_end,
            "Start Formatted": format_time(shift_start),
            "End Formatted": format_time(shift_end),
            "Shift Duration": ROUT_TIME,
        })

        current_time = shift_end + DRIVE_CHANGE_TIME

        if current_time - START_TIME >= max_hours:
            break

    return schedule

# Генерация полного расписания
def generate_full_schedule():
    schedules = []
    for bus_id in range(1, NUM_BUSES + 1):
        driver_type = "A" if bus_id % 2 == 0 else "B"
        schedules.extend(generate_bus_schedule(bus_id, driver_type))
    return pd.DataFrame(schedules)

# Оценка расписания
def evaluate_schedule(schedule):
    morning_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[0])
    evening_peak_buses = count_buses_in_peak(schedule, PEAK_HOURS[1])
    return morning_peak_buses + evening_peak_buses

# Генетический алгоритм
def genetic_algorithm(iterations=100, population_size=10):
    full_schedule = generate_full_schedule()
    best_schedule = None
    best_score = -1

    for _ in range(iterations):
        population = [
            full_schedule.sample(frac=1).reset_index(drop=True) for _ in range(population_size)
        ]

        for schedule in population:
            score = evaluate_schedule(schedule)
            if score > best_score:
                best_schedule = schedule
                best_score = score

    return best_schedule, best_score

# Алгоритм в лоб
def brute_force_schedule():
    full_schedule = generate_full_schedule()
    best_schedule = None
    best_score = -1

    for _ in range(100):  # Перебор 100 случайных вариантов
        random_schedule = full_schedule.sample(frac=1).reset_index(drop=True)
        score = evaluate_schedule(random_schedule)

        if score > best_score:
            best_schedule = random_schedule
            best_score = score

    return best_schedule, best_score

# Сравнение алгоритмов
def compare_algorithms():
    print("Запуск генетического алгоритма...")
    start_time = time.time()
    genetic_result, genetic_score = genetic_algorithm()
    genetic_time = time.time() - start_time
    print(f"Генетический алгоритм: {genetic_score} автобусов в часы пик, время выполнения: {genetic_time:.4f} сек")

    print("\nЗапуск алгоритма в лоб...")
    start_time = time.time()
    brute_force_result, brute_force_score = brute_force_schedule()
    brute_force_time = time.time() - start_time
    print(f"Алгоритм в лоб: {brute_force_score} автобусов в часы пик, время выполнения: {brute_force_time:.4f} сек")

    # Сравнение
    print("\nСравнение результатов:")
    print(f"Генетический алгоритм: {genetic_score} (время: {genetic_time:.4f} сек)")
    print(f"Алгоритм в лоб: {brute_force_score} (время: {brute_force_time:.4f} сек)")

# Запуск сравнения
compare_algorithms()


Запуск генетического алгоритма...
Генетический алгоритм: 20 автобусов в часы пик, время выполнения: 5.2575 сек

Запуск алгоритма в лоб...
Алгоритм в лоб: 20 автобусов в часы пик, время выполнения: 0.6414 сек

Сравнение результатов:
Генетический алгоритм: 20 (время: 5.2575 сек)
Алгоритм в лоб: 20 (время: 0.6414 сек)
